In [1]:
import pandas as pd
import numpy as np

In [189]:
data_posts = pd.read_csv('kawo-data-science-challenge/KAWO-data-science-raw-data/2016_cny_posts.csv', error_bad_lines=False)
data_hourly = pd.read_csv('kawo-data-science-challenge/KAWO-data-science-raw-data/2016_cny_hourly.csv', error_bad_lines=False)

In [18]:
len(data_posts['screen_name'].unique())

927

In [28]:
data_posts.shape

(167726, 11)

In [20]:
data_posts.columns

Index([u'screen_name', u'created', u'text', u'reposts', u'comments', u'likes',
       u'pictures', u'hasgif', u'links', u'video', u'url'],
      dtype='object')

In [25]:
len(data_hourly['screen_name'].unique())

1020

In [32]:
data_posts['created'] = pd.to_datetime(data_posts['created'], errors='ignore')

## Change "created" to datetime object and remove outlier

In [65]:
created_time_list = []
boolen_list = []

for time in data_posts['created'].tolist():
    try:
        time_temp = pd.to_datetime(time)
        created_time_list.append(time_temp)
        boolen_list.append(True)
    except ValueError:
        # print(time)
        created_time_list.append(time)
        boolen_list.append(False)

In [71]:
data_posts['created_datetime'] = created_time_list
data_posts = data_posts[boolen_list]
data_posts.drop('created', axis=1, inplace=True)

In [248]:
data_posts.dropna(subset=['created_datetime'], inplace=True)

In [250]:
data_posts['created_datetime'] = data_posts['created_datetime'].apply(lambda x: x.ceil('60min'))

In [251]:
time_df_temp = data_posts['created_datetime'].apply(lambda x: (x.hour, x.minute)).apply(pd.Series)
time_df_temp.rename(columns={0: 'hour', 1: 'minute'}, inplace=True)

In [254]:
# data_posts['hour'] = time_df_temp['hour'].tolist()
# data_posts['minute'] = time_df_temp['minute'].tolist()

In [101]:
data_posts = pd.merge(data_posts, time_df_temp, left_index=True, right_index=True, how='left')

## Text parsing

### word count

In [116]:
data_posts['word_count'] = data_posts['text'].apply(lambda x: round(len(x)/3.) if isinstance(x, str) else 1.)

### emoji?

In [137]:
data_posts['has_emoji'] = data_posts['text'].apply(lambda x: int(('[' in x) & (']' in x)) if isinstance(x, str) else 0)

### hash tag

In [142]:
data_posts['has_tag'] = data_posts['text'].apply(lambda x: int('#' in x) if isinstance(x, str) else 0)

### @ someone?

In [149]:
data_posts['has_@'] = data_posts['text'].apply(lambda x: int('@' in x) if isinstance(x, str) else 0)

# Data Hourly

### Fans Increase

In [191]:
data_hourly_sort = data_hourly.sort(['screen_name', 'savedate'])

/Users/haoliu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [192]:
fans_increase = np.array(data_hourly_sort['fans_count'][1: ]) - np.array(data_hourly_sort['fans_count'][0: -1])
fans_increase = np.hstack((0, fans_increase))
data_hourly_sort['fans_increase'] = fans_increase

In [194]:
name_change_boolen = np.array(data_hourly_sort['screen_name'][1: ]) == np.array(data_hourly_sort['screen_name'][0: -1])
name_change_boolen = np.hstack((False, name_change_boolen))
data_hourly_sort['name_change'] = name_change_boolen 

In [206]:
data_hourly_sort = data_hourly_sort.query('name_change == True')
data_hourly_sort.drop(['fans_count', 'name_change'], axis=1, inplace=True)

In [211]:
data_hourly_sort['savedate'] = pd.to_datetime(data_hourly_sort['savedate'])

/Users/haoliu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### Concatenate posts data and hourly data

### Videio

In [302]:
data_merged['has_video'] = map(lambda x: int(type(x) == str), np.array(data_merged['video']))

In [260]:
data_merged = pd.merge(data_posts, data_hourly_sort, left_on=['screen_name', 'created_datetime'], right_on=['screen_name', 'savedate'], how='left')



### Drop columns and NA rows

In [270]:
data_merged.drop(['text', 'links', 'url', 'created_datetime', 'savedate'], axis=1, inplace=True)
data_merged.drop(['comments_count', 'retweets_count', 'likes_count'], axis=1, inplace=True)
data_merged.drop('video', axis=1, inplace=True)
data_merged.dropna(inplace=True)

### create target

In [313]:
data_merged.head()

,screen_name,reposts,comments,likes,pictures,hasgif,hour,minute,word_count,has_emoji,has_tag,has_@,fans_increase,has_video
0,360杀毒,6,4,10.0,0.0,0,21,0,3.0,1,0,0,-5.0,0
1,360杀毒,0,0,2.0,0.0,0,12,0,15.0,0,0,0,0.0,0
2,360杀毒,2,4,8.0,0.0,0,5,0,2.0,1,0,0,-1.0,0
3,360杀毒,2,5,16.0,0.0,0,12,0,5.0,1,0,0,0.0,0
4,360杀毒,0,6,8.0,0.0,0,16,0,10.0,1,0,0,-1.0,0


In [325]:
data_merged['target'] = data_merged.loc[:, ['reposts', 'comments', 'likes', 'fans_increase']].astype(float).sum(axis=1)

In [327]:
data_merged.to_csv('data_cleaned.csv')